# Loading Libraries  

In [38]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

In [39]:
username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2
    complePath = '/home/alinux/unibg/git_repo/SCGProject/Datasets/'

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")

dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(',','.')

Username:  marcovinciguerra


# Verifica pezzi intermedi

# Analisi qtà articoli a BUDGET

In [40]:
dfConsumi.rename(columns = {'Budget/cons':'status'}, inplace = True)
qtaArtBudget = sqldf('select NrArticolo, count(*) as qta from dfConsumi where status = "BUDGET" group by NrArticolo')

dfVendite.rename(columns = {'budget/cons':'status'}, inplace = True)
qtaArtVendBudget = sqldf('select NrArticolo, count(*) as qta from dfVendite where status = "BUDGET" group by NrArticolo')

artProdeVendBudget = sqldf('select P.NrArticolo from qtaArtBudget P join qtaArtVendBudget V on P.NrArticolo = V.NrArticolo')



consumiB = sqldf('select * from dfConsumi where status = "BUDGET"')


# ho trovato articoli che hanno consumato risorse a budget ma non sono stati messi tra le vendite a budget
artProdeNONVendBudget = sqldf('select * from consumiB group by NrArticolo having NrArticolo not in (select NrArticolo from artProdeVendBudget)')
print(artProdeNONVendBudget)

#print(sqldf('select NrArticolo from artProdeVendBudget'))

# prendo gli articoli che usano come materia prima almeno un articolo prodotto internamente
# sqldf('select NrMovimento, status, C.NrArticolo, CodiceMP, NrDocumento, QuantityMPImpiegata, ImportoCostoTOTALE from consumiB C join artProdeNONVendBudget A on C.CodiceMP = A.NrArticolo')

costoPerArticoloBudget = sqldf('select NrArticolo, sum(ImportoCostoTOTALE) as costoMpPerArt from consumiB where NrArticolo not in (select NrArticolo from artProdeNONVendBudget) group by NrArticolo')
sqldf('select sum(costoMpPerArt) as costoTotBudget from costoPerArticoloBudget')

    Unnamed: 0  NrMovimento  status    CodiceMP  NrArticolo  NrDocumento  \
0         2616        25670  BUDGET  ART0005190  ART0001391  ODP17-02529   
1         2618        27664  BUDGET  ART0001078  ART0001409  ODP17-05018   
2         2630        32354  BUDGET  ART0005681  ART0001429  ODP17-09820   
3         2653        25529  BUDGET  ART0005227  ART0001514  ODP17-02757   
4         2805        27095  BUDGET  ART0005227  ART0001761  ODP17-03730   
..         ...          ...     ...         ...         ...          ...   
86        3802        35396  BUDGET  ART0005403  ART0004015  ODP17-14587   
87        4027        26455  BUDGET  ART0005426  ART0004149  ODP17-03082   
88        4030        26347  BUDGET  ART0005537  ART0004152  ODP17-01270   
89        4037        24314  BUDGET  ART0005423  ART0004175  ODP16-16946   
90        4040        23600  BUDGET  ART0005148  ART0004202  ODP17-01566   

    QuantityMPImpiegata  ImportoCostoTOTALE  
0                 234.0               22.

,costoTotBudget
0,172660.4


# Includo il prezzo processi

In [41]:
status = "BUDGET"
impRisBudget = sqldf('select * from dfImpiegoRisorse where budgetConsuntivo = "BUDGET"')

impRisBudgetConPzuOrar = sqldf('select * from impRisBudget I join dfCostoOrarioBudget C on I.Risorsa = C.Risorsa and I.NrAreaProduzione = C.AreaProduzione')
impCostoProcBudget = sqldf('select NrArticolo, AreaProduzione, Risorsa, (TempoRisorsa*CostoOrario) as costoProc from impRisBudgetConPzuOrar')

impCostoProcPerArticoloBudget = sqldf('select NrArticolo, sum(costoProc) as costoProc from impCostoProcBudget group by NrArticolo')
#faccio il totale dei processi per articolo e il totale di mp per articolo da consumi e gli metto insieme

costoTotPerArticoloBudget = sqldf('select A.NrArticolo, costoProc+costoMpPerArt as costoTot from impCostoProcPerArticoloBudget A join costoPerArticoloBudget B on A.NrArticolo=B.NrArticolo')
print(costoTotPerArticoloBudget)

#aggiungo al costo del singolo articolo, il costo del articolo usato come materia prima
#costoTotPerArticoloBudgetConMatIndiretti = sqldf('select ')

     NrArticolo   costoTot
0    ART0000018   111.4000
1    ART0000041   385.2500
2    ART0000042   537.5000
3    ART0000043   376.0668
4    ART0000115   753.1900
..          ...        ...
408  ART0005109   108.9700
409  ART0005110    74.0000
410  ART0005111    90.3750
411  ART0005112   121.6250
412  ART0005136  5758.3550

[413 rows x 2 columns]


# Cerco di calcolare il prezzo di vendita a budget

In [42]:
venditeB = sqldf('select * from dfVendite where status = "BUDGET"')
venditeC = sqldf('select * from dfVendite where status = "CONSUNTIVO" or status = "Consuntivo"')

cambioB = sqldf('select * from dfCambio where Anno = "BUDGET"')
valutaCliente = sqldf('select Nr, TassoCambioMedio from dfClienti C join cambioB CB on C.Valuta = CB.CodiceValuta ')

clienteConValutaB = sqldf('select * from venditeB B join valutaCliente VC on B.NrOrigine = Nr')
clienteConValutaC = sqldf('select * from venditeC B join valutaCliente VC on B.NrOrigine = Nr')

venditeBinEuro = sqldf('select NrMovimento, status, NrArticolo, NrOrigine, Quantity, ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio as PzuEuro, TassoCambioMedio from clienteConValutaB')
x = sqldf('select sum(PzuEuro) from  venditeBinEuro')
print(x)
print(venditeBinEuro)

venditeCinEuro = sqldf('select NrMovimento, status, NrArticolo, NrOrigine, Quantity, ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio as PzuEuro, TassoCambioMedio from clienteConValutaC')

pzuPerArticoloBudget = sqldf('select NrArticolo, sum(PzuEuro) as pzuEuroPerArticolo from venditeBinEuro group by NrArticolo')
#sqldf('select sum(pzuEuroPerArticolo) as ricaviBudget from pzuPerArticoloBudget')


totdiff = sqldf('select A.NrArticolo, (pzuEuroPerArticolo-costoTot) as diff from costoTotPerArticoloBudget A join pzuPerArticoloBudget B on A.NrArticolo=B.NrArticolo')

sqldf('select sum(diff) from totdiff') #sembra ragionevole
#todo sotrarre dal pzuEuroPerArticolo il costo per la lavorazione interna di articoli che sono stati usati come MP per altri prodotti


    sum(PzuEuro)
0  395335.892443
      NrMovimento  status  NrArticolo NrOrigine  Quantity     PzuEuro  \
0           21621  BUDGET  ART0001433    C00140         4   17.835120   
1           21629  BUDGET  ART0001610    C00140        12  137.823736   
2           21631  BUDGET  ART0003269    C00140        24  886.054454   
3           21633  BUDGET  ART0002249    C00140         1   34.465421   
4           21634  BUDGET  ART0002370    C00140         1   62.347026   
...           ...     ...         ...       ...       ...         ...   
1204        35464  BUDGET  ART0004363    C00004       116   15.792545   
1205        35466  BUDGET  ART0000643    C00004        32    2.593193   
1206        35468  BUDGET  ART0004440    C00004         5    5.672609   
1207        35959  BUDGET  ART0001759    C00140         2  132.814723   
1208        40190  BUDGET  ART0004169    C00297         2    2.106969   

     TassoCambioMedio  
0              1.0541  
1              1.0541  
2              1.

,sum(diff)
0,103510.849143


### Sottrago dai ricavi anche il costo di produzione dei articoli usati come materia prima per prodotti finali

dovrei prendere i 90 articoli che ho prodotto ma non ho venduto, calcolare il loro costo della materia prima e i costi di lavorazione dopodiche quel costo lo devo aggiungere al costo di produzione del articolo che usa come mp l'articolo di sopra

In [43]:
# articoli da capire dove sono finiti, sono state consumate risorse per farli ma non sono stati utilizzati per la produzione di un'altro articolo oppure venduti

#verifico che siano effettivamente loro
#x = sqldf('select distinct CodiceMP from prodInternaPerArticoliBudget group by CodiceMP')
#articoliDaCapireDoveSonoFiniti = sqldf('select NrArticolo from artProdeNONVendBudget where NrArticolo not in (select CodiceMP from x)')

#sqldf('select * from consumiB C1 join prodInternaPerArticoliBudget C2 on C1.CodiceMP = C2.NrArticolo')



In [44]:
sqldf('''
select NrArticolo, sum(ImportoCostoTOTALE)
from consumiB
group by codiceMP
''')

,NrArticolo,sum(ImportoCostoTOTALE)
0,ART0002628,2.59
1,ART0000018,203.48
2,ART0002866,158.70
3,ART0003984,0.58
4,ART0001409,5.47
...,...,...
549,ART0001731,90.00
550,ART0001731,0.00
551,ART0001770,148.05
552,ART0004363,826.57


In [45]:

y = sqldf('''select codiceMP, sum(ImportoCostoTOTALE) as tot
         from consumiB
         group by codiceMP
''')

costoTotMPBudget = sqldf('''select sum(tot)
             from y
''')
costoTotMPBudget

,sum(tot)
0,192906.47


In [46]:
consumiC = sqldf('''select *
                    from dfConsumi
                    where status = "CONSUNTIVO"
''')
costoTotMPCons = sqldf('''select codiceMP, sum(ImportoCostoTOTALE) as tot
         from consumiC
         group by codiceMP
''')

costoTotMPCons = sqldf('''select sum(tot)
             from costoTotMPCons
''')
costoTotMPCons

,sum(tot)
0,289639.5


In [47]:
costoTotMPCons-costoTotMPBudget

,sum(tot)
0,96733.03


In [48]:

y = sqldf('''select codiceMP, count(codiceMP) as tot
         from consumiB
         group by codiceMP
''')

y

,CodiceMP,tot
0,ART0001073,1
1,ART0001074,6
2,ART0001076,1
3,ART0001077,1
4,ART0001078,1
...,...,...
549,ART0005943,1
550,ART0005944,1
551,ART0005951,1
552,ART0005953,1


In [49]:
y = sqldf('''select codiceMP, count(codiceMP) as tot
         from consumiC
         group by codiceMP
''')

y

,CodiceMP,tot
0,ART0000771,1
1,ART0000809,1
2,ART0001066,1
3,ART0001068,1
4,ART0001073,3
...,...,...
625,ART0007130,1
626,ART0007133,1
627,ART0007141,1
628,ART0007148,1


### mix valuta standard

In [50]:


countDollar = sqldf('''
    select count(*)
    from clienteConValutaB
    where TassoCambioMedio = 1.0541
''')
countDollar

countEuro = sqldf('''
    select count(*)
    from clienteConValutaB
    where TassoCambioMedio = 1
''')
countEuro

countYen = sqldf('''
    select count(*)
    from clienteConValutaB
    where TassoCambioMedio = 123.4
''')
countYen

totmixvaluta = countDollar+countEuro+countYen

mixdollarB = countDollar/totmixvaluta
mixeuroB = countEuro/totmixvaluta
mixyenB = countYen/totmixvaluta

print(mixdollarB)
print(mixeuroB)
print(mixyenB)

   count(*)
0  0.769231
   count(*)
0  0.100083
   count(*)
0  0.130687


In [51]:
#venditeB = sqldf('select * from dfVendite where status = "CONSUNTIVO"')
cambioC = sqldf('select * from dfCambio where Anno = "CONSUNTIVO"')

valutaClienteC = sqldf('select Nr, TassoCambioMedio from dfClienti C join cambioC CB on C.Valuta = CB.CodiceValuta ')

clienteConValutaC = sqldf('select * from venditeB B join valutaClienteC VC on B.NrOrigine = Nr')


countDollar = sqldf('''
    select count(*)
    from clienteConValutaC
    where TassoCambioMedio = 1.1993
''')
countDollar

countEuro = sqldf('''
    select count(*)
    from clienteConValutaC
    where TassoCambioMedio = 1
''')
countEuro

countYen = sqldf('''
    select count(*)
    from clienteConValutaC
    where TassoCambioMedio = 135.01
''')
countYen

totmixvaluta = countDollar+countEuro+countYen

mixdollarC = countDollar/totmixvaluta
mixeuroC = countEuro/totmixvaluta
mixyenC = countYen/totmixvaluta

print(countDollar)
print(countEuro)
print(countYen)

print(mixdollarC)
print(mixeuroC)
print(mixyenC)

clienteConValutaC


   count(*)
0       930
   count(*)
0       121
   count(*)
0       158
   count(*)
0  0.769231
   count(*)
0  0.100083
   count(*)
0  0.130687


,Unnamed: 0,NrMovimento,status,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,Nr,TassoCambioMedio
0,0,21621,BUDGET,ART0001433,C00140,4,18.80,C00140,1.1993
1,1,21629,BUDGET,ART0001610,C00140,12,145.28,C00140,1.1993
2,2,21631,BUDGET,ART0003269,C00140,24,933.99,C00140,1.1993
3,3,21633,BUDGET,ART0002249,C00140,1,36.33,C00140,1.1993
4,4,21634,BUDGET,ART0002370,C00140,1,65.72,C00140,1.1993
...,...,...,...,...,...,...,...,...,...
1204,1205,35464,BUDGET,ART0004363,C00004,116,1948.80,C00004,135.01
1205,1206,35466,BUDGET,ART0000643,C00004,32,320.00,C00004,135.01
1206,1207,35468,BUDGET,ART0004440,C00004,5,700.00,C00004,135.01
1207,1214,35959,BUDGET,ART0001759,C00140,2,140.00,C00140,1.1993


In [52]:
qb = sqldf('select codiceMP, sum(QuantityMPImpiegata) as qmpB from consumiB group by codiceMP')
qc = sqldf('select codiceMP, sum(QuantityMPImpiegata) as qmpC from consumiC group by codiceMP')
sqldf('select B.codiceMP, B.qmpB, C.qmpC from qb B join qc C on B.codiceMP=C.codiceMP ')

,CodiceMP,qmpB,qmpC
0,ART0001073,30.0,296.0
1,ART0001074,1185.0,1478.0
2,ART0001076,460.0,2449.0
3,ART0001078,263.0,2274.0
4,ART0001080,620.0,1350.0
...,...,...,...
466,ART0005934,1890.0,43900.0
467,ART0005944,1000.0,880.0
468,ART0005951,1575.0,11016.0
469,ART0005953,116.0,4.0


In [53]:
x = sqldf('''select distinct NrArticolo
         from venditeB
         where NrArticolo in (select distinct NrArticolo from venditeC)
 ''')

y = sqldf('''
        select *
        from consumiB C join x on C.NrArticolo = x.NrArticolo 
''')


In [54]:
z =sqldf('''
        select *
        from consumiC C join x on C.NrArticolo = x.NrArticolo 
''')

# Controllo quali degli articoli intermedi ho venduto a BUDGET

In [55]:
mpB = sqldf('''select NrMovimento,status,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiB group by NrArticolo''')
#A.NrArticolo, A.CodiceMP, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE, A.CodiceMP
prodInternaB = sqldf('''select A.NrMovimento, A.status, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiB A join mpB B on A.CodiceMP = B.NrArticolo''')
prodInternaB


,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,24540,BUDGET,ART0001868,ART0000748,ODP17-02386,49.0,1278.86
1,28241,BUDGET,ART0001868,ART0000748,ODP17-05535,50.0,1348.54
2,34074,BUDGET,ART0001868,ART0000748,ODP17-10871,50.0,1308.66
3,35495,BUDGET,ART0001868,ART0000748,ODP17-15078,50.0,1311.31
4,32324,BUDGET,ART0001900,ART0000749,ODP17-09769,2.0,376.41
...,...,...,...,...,...,...,...
291,35359,BUDGET,ART0004149,ART0004104,ODP17-13148,4.0,17.76
292,35362,BUDGET,ART0004152,ART0004104,ODP17-13148,4.0,65.44
293,22678,BUDGET,ART0004149,ART0004146,ODP16-13864,31.0,161.44
294,22681,BUDGET,ART0004152,ART0004146,ODP16-13864,31.0,366.68


In [56]:
mpIntermediB = sqldf('''select distinct CodiceMP from prodInternaB''')
mpIntermediB

#sqldf('''select * from consumiB A join mpIntermedi B on A.CodiceMP = B.CodiceMP ''')
#sqldf('''select * from prodInterna A join artProdeVendBudget B on A.CodiceMP = B.NrArticolo''')

#prodInterna

,CodiceMP
0,ART0001868
1,ART0001900
2,ART0001789
3,ART0001921
4,ART0001811
...,...
79,ART0001767
80,ART0004149
81,ART0004152
82,ART0004015


In [57]:
artProdeNONVendBudget

,Unnamed: 0,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,2616,25670,BUDGET,ART0005190,ART0001391,ODP17-02529,234.0,22.75
1,2618,27664,BUDGET,ART0001078,ART0001409,ODP17-05018,263.0,5.47
2,2630,32354,BUDGET,ART0005681,ART0001429,ODP17-09820,222.0,7.35
3,2653,25529,BUDGET,ART0005227,ART0001514,ODP17-02757,90.0,13.23
4,2805,27095,BUDGET,ART0005227,ART0001761,ODP17-03730,180.0,27.77
...,...,...,...,...,...,...,...,...
86,3802,35396,BUDGET,ART0005403,ART0004015,ODP17-14587,200.0,52.29
87,4027,26455,BUDGET,ART0005426,ART0004149,ODP17-03082,3745.0,5.38
88,4030,26347,BUDGET,ART0005537,ART0004152,ODP17-01270,4260.0,14.20
89,4037,24314,BUDGET,ART0005423,ART0004175,ODP16-16946,40.0,74.67


In [58]:
venditeBinEuro

,NrMovimento,status,NrArticolo,NrOrigine,Quantity,PzuEuro,TassoCambioMedio
0,21621,BUDGET,ART0001433,C00140,4,17.835120,1.0541
1,21629,BUDGET,ART0001610,C00140,12,137.823736,1.0541
2,21631,BUDGET,ART0003269,C00140,24,886.054454,1.0541
3,21633,BUDGET,ART0002249,C00140,1,34.465421,1.0541
4,21634,BUDGET,ART0002370,C00140,1,62.347026,1.0541
...,...,...,...,...,...,...,...
1204,35464,BUDGET,ART0004363,C00004,116,15.792545,123.4
1205,35466,BUDGET,ART0000643,C00004,32,2.593193,123.4
1206,35468,BUDGET,ART0004440,C00004,5,5.672609,123.4
1207,35959,BUDGET,ART0001759,C00140,2,132.814723,1.0541


In [59]:
c = sqldf('''
    select B.NrMovimento, B.status, B.NrArticolo, B.NrOrigine, B.Quantity, B.PzuEuro
    from mpIntermediB A join venditeBinEuro B on A.CodiceMP = B.NrArticolo''')
c

,NrMovimento,status,NrArticolo,NrOrigine,Quantity,PzuEuro
0,35959,BUDGET,ART0001759,C00140,2,132.814723
1,33146,BUDGET,ART0001981,C00140,1,22.293900
2,34271,BUDGET,ART0002070,C00140,1,124.845840


In [60]:
sommaBudget = sqldf("SELECT SUM(PzuEuro) FROM c ")
sommaBudget

,SUM(PzuEuro)
0,279.954464


### Consumi MP senza articoli intermedi per il BUDGET

In [61]:
consumiBNOArtInterm = sqldf('''select * from consumiB where NrArticolo not in (select CodiceMP from mpIntermediB)''')
consumiBNOArtInterm

,Unnamed: 0,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
2,4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00
3,6,28852,BUDGET,ART0001074,ART0000043,ODP17-05342,210.0,36.06
4,8,28514,BUDGET,ART0005683,ART0000115,ODP17-04831,645.0,210.48
...,...,...,...,...,...,...,...,...
1516,4280,34378,BUDGET,ART0005176,ART0005136,ODP17-14994,6.0,0.00
1517,4281,34832,BUDGET,ART0005176,ART0005136,ODP17-13850,3.0,0.00
1518,4282,34834,BUDGET,ART0005928,ART0005136,ODP17-13850,2.0,0.00
1519,4283,34837,BUDGET,ART0005172,ART0005136,ODP17-15077,250.0,4.16


# Stesso discorso a CONSUNTIVO

In [62]:
consumiC = sqldf('''select * from dfConsumi where status = "CONSUNTIVO" or status = "Consuntivo"''')
mpC = sqldf('''select NrMovimento,status,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiC group by NrArticolo''')
#A.NrArticolo, A.CodiceMP, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE, A.CodiceMP
prodInternaC= sqldf('''select A.NrMovimento, A.status, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiC A join mpC B on A.CodiceMP = B.NrArticolo''')
prodInternaC

,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,38877,CONSUNTIVO,ART0001868,ART0000748,ODP18-02202,50.0,1205.91
1,41018,CONSUNTIVO,ART0001868,ART0000748,ODP18-03096,40.0,811.05
2,42368,CONSUNTIVO,ART0001868,ART0000748,ODP18-05151,20.0,535.63
3,43917,CONSUNTIVO,ART0001868,ART0000748,ODP18-05148,30.0,615.54
4,44464,CONSUNTIVO,ART0001868,ART0000748,ODP18-05803,40.0,824.86
...,...,...,...,...,...,...,...
530,61499,CONSUNTIVO,ART0004152,ART0004146,ODP19-03920,10.0,163.60
531,44874,CONSUNTIVO,ART0004149,ART0005136,ODP18-05899,1.0,1.73
532,44877,CONSUNTIVO,ART0004152,ART0005136,ODP18-05899,1.0,16.36
533,56916,CONSUNTIVO,ART0001772,ART0005136,ODP18-06023,2.0,118.46


In [63]:
mpIntermediC = sqldf('''select distinct CodiceMP from prodInternaC''')
mpIntermediC

,CodiceMP
0,ART0001868
1,ART0001900
2,ART0001789
3,ART0001921
4,ART0001811
...,...
81,ART0003940
82,ART0004015
83,ART0004149
84,ART0004152


In [64]:

sx = sqldf('''
    select B.NrMovimento, B.status, B.NrArticolo, B.NrOrigine, B.Quantity, B.PzuEuro
    from mpIntermediC A join venditeCinEuro B on A.CodiceMP = B.NrArticolo''')

sqldf(''' select sum(PzuEuro) from sx''')

,sum(PzuEuro)
0,8689.700847


In [65]:
consumiCNOArtInterm = sqldf('''select * from consumiC where NrArticolo not in (select CodiceMP from mpIntermediC)''')
consumiCNOArtInterm

,Unnamed: 0,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
1,3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
2,5,66817,CONSUNTIVO,ART0005697,ART0000042,ODP19-05998,4.0,100.00
3,7,66820,CONSUNTIVO,ART0001074,ART0000043,ODP19-05998,208.0,35.71
4,10,57633,CONSUNTIVO,ART0005683,ART0000115,ODP19-00633,651.0,268.26
...,...,...,...,...,...,...,...,...
2540,4514,66172,CONSUNTIVO,ART0005850,ART0005136,ODP19-02057,155.0,1.23
2541,4515,66235,CONSUNTIVO,ART0006245,ART0005136,ODP19-02528,700.0,0.00
2542,4516,66446,CONSUNTIVO,ART0006381,ART0005136,ODP19-02674,300.0,1.62
2543,4517,66594,CONSUNTIVO,ART0006345,ART0005136,ODP19-06688,9.0,0.00


# UNIONE DELLE DUE TABELLE PER FARE IL NUOVO DATASET

In [79]:
dfConsumiNuovo = sqldf('''  SELECT *
                            FROM consumiBNOArtInterm
                            UNION
                            SELECT *
                            FROM consumiCNOArtInterm
                            ''')
dfConsumiNuovo


,Unnamed: 0,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00
...,...,...,...,...,...,...,...,...
4061,4514,66172,CONSUNTIVO,ART0005850,ART0005136,ODP19-02057,155.0,1.23
4062,4515,66235,CONSUNTIVO,ART0006245,ART0005136,ODP19-02528,700.0,0.00
4063,4516,66446,CONSUNTIVO,ART0006381,ART0005136,ODP19-02674,300.0,1.62
4064,4517,66594,CONSUNTIVO,ART0006345,ART0005136,ODP19-06688,9.0,0.00


In [80]:
#Vecchio dataset
dfConsumi #è giusto che sia molto più grande

,Unnamed: 0,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00
...,...,...,...,...,...,...,...,...
4514,4514,66172,CONSUNTIVO,ART0005850,ART0005136,ODP19-02057,155.0,1.23
4515,4515,66235,CONSUNTIVO,ART0006245,ART0005136,ODP19-02528,700.0,0.00
4516,4516,66446,CONSUNTIVO,ART0006381,ART0005136,ODP19-02674,300.0,1.62
4517,4517,66594,CONSUNTIVO,ART0006345,ART0005136,ODP19-06688,9.0,0.00


## Budget

Lavorazioni interne

In [66]:
impiegoConsuntivo = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO' or budgetConsuntivo = 'Consuntivo'")
print(impiegoConsuntivo.shape)
#join con i costi orari
impiegoConsuntivo  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoConsuntivo as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoConsuntivo.shape)
# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoConsuntivoOld = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivo1 = impiegoConsuntivoOld.copy()
# ASSUNZIONE: la quantity di output maggiore tiene conto di tutte quelle inferiori.


(3791, 9)
(3723, 9)


In [67]:
impiegoConsuntivoNew = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, max(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivoNew

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0
2,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0
3,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,4.0
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0
...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,ODP19-06532,Fresatura,A30,RIS14,1.2500,50,3.0
3013,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0000,15,4.0
3014,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5000,80,4.0
3015,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0000,15,9.0


In [68]:
print(sqldf('select sum(QuantitydiOutput) from impiegoConsuntivoOld'))
print(sqldf('select sum(QuantitydiOutput) from impiegoConsuntivoNew'))

   sum(QuantitydiOutput)
0                25950.0
   sum(QuantitydiOutput)
0                25327.0


In [69]:
sqldf('select NrArticolo, NrOrdineProduzione, count(*) as c from impiegoConsuntivoNew group by NrArticolo, NrOrdineProduzione order by c')

,NrArticolo,NrOrdineProduzione,c
0,ART0000116,ODP18-03472,1
1,ART0000116,ODP18-05889,1
2,ART0000119,ODP19-04430,1
3,ART0000121,ODP18-03469,1
4,ART0000128,ODP18-03529,1
...,...,...,...
1236,ART0005136,ODP19-01954,5
1237,ART0000043,ODP19-05998,6
1238,ART0001731,ODP18-04233,6
1239,ART0002360,ODP19-06224,6


In [70]:
impiegoBudget = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget'")
#join con i costi orari
print(impiegoBudget.shape)
impiegoBudget = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoBudget as i join dfCostoOrarioBudget as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoBudget.shape)
# righe a consuntivo sono di più che a budget e la somma dei 2 non corrisponde al numero di righe della tabella impiego risorse

# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoBudgetOld = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoBudget1 = impiegoBudgetOld.copy()

impiegoBudgetNew = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, max(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')


(2473, 9)
(2450, 9)


In [71]:
print(sqldf('select sum(QuantitydiOutput) from impiegoBudgetOld'))
print(sqldf('select sum(QuantitydiOutput) from impiegoBudgetNew'))

   sum(QuantitydiOutput)
0                15768.5
   sum(QuantitydiOutput)
0               15580.25


In [72]:
impiegoBudgetNew = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudgetNew
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')

In [73]:
impiegoBudgetNew

sqldf('''select NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudgetNew 
group by NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

sqldf('''select NrArticolo, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudgetNew 
group by NrArticolo, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

,NrArticolo,Descrizione,NrAreaProduzione,Risorsa,count(*)
0,ART0005136,Controllo qualità,CQ,RIS16,19
1,ART0005136,Tornitura,A20,RIS2,13
2,ART0000795,Controllo qualità,CQ,RIS16,11
3,ART0000762,Controllo qualità,CQ,RIS16,9
4,ART0000782,Controllo qualità,CQ,RIS16,8
...,...,...,...,...,...
1395,ART0005136,Brocciatura / stozzatrice,A33,RIS2,1
1396,ART0005136,Foratura/Maschiatura,A32,RIS9,1
1397,ART0005136,Fresatura,A30,RIS1,1
1398,ART0005136,Fresatura,A30,RIS13,1


#### Group by CodiceMP

se non facciamo qualche group by a priori si sdoppiano le righe, proviamo prima con CodiceMP

In [74]:
print(sqldf('select sum(QuantityMPImpiegata) as Volume from consumiConsuntivoUnit'))
print(sqldf('select sum(QuantityMPImpiegata) as Volume from consumiBudgetUnit'))

PandaSQLException: (sqlite3.OperationalError) no such table: consumiConsuntivoUnit
[SQL: select sum(QuantityMPImpiegata) as Volume from consumiConsuntivoUnit]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
consumiConsuntivoUnitByMP = sqldf('''select CodiceMP, NrArticolo,938999.16 as Volume, sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario , sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
from consumiConsuntivoUnit      
group by CodiceMP, NrArticolo''')
ConsumiBudgetUnitByMP = sqldf('''select CodiceMP, NrArticolo,550834.0 as Volume, sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario , sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
from consumiBudgetUnit      
group by CodiceMP, NrArticolo''')
consumiConsuntivoUnitByMP

,CodiceMP,NrArticolo,Volume,QuantityMPImpiegata,CostoUnitario,ImportoCostoTOTALE
0,ART0000771,ART0005136,938999.16,1.0,230.950000,230.95
1,ART0000809,ART0005136,938999.16,2.0,561.350000,1122.70
2,ART0001066,ART0003099,938999.16,1064.0,0.001870,1.99
3,ART0001068,ART0004671,938999.16,190.0,0.036000,6.84
4,ART0001073,ART0002240,938999.16,184.0,0.086685,15.95
...,...,...,...,...,...,...
1074,ART0007130,ART0001228,938999.16,15.0,1.500000,22.50
1075,ART0007133,ART0005136,938999.16,3.0,0.000000,0.00
1076,ART0007141,ART0002191,938999.16,2.0,6.500000,13.00
1077,ART0007148,ART0002619,938999.16,2.0,30.000000,60.00


In [ ]:
conVol = sqldf('select CodiceMP, sum(QuantityMPImpiegata) as VolumeByMP from consumiConsuntivoUnitByMP group by CodiceMP')
buVol = sqldf('select CodiceMP, sum(QuantityMPImpiegata) as VolumeByMP from ConsumiBudgetUnitByMP group by CodiceMP')
buVol

,CodiceMP,VolumeByMP
0,ART0001073,30.0
1,ART0001074,1185.0
2,ART0001076,460.0
3,ART0001077,46.0
4,ART0001078,263.0
...,...,...
549,ART0005943,20.0
550,ART0005944,1000.0
551,ART0005951,1575.0
552,ART0005953,116.0


In [ ]:
consumiConsuntivoUnitByMP = sqldf('''select CodiceMP, NrArticolo, VolumeByMP, (QuantityMPImpiegata/VolumeByMP) as Impiego, CostoUnitario, ImportoCostoTOTALE
from consumiConsuntivoUnitByMP natural join conVol ''')
ConsumiBudgetUnitByMP = sqldf('''select CodiceMP, NrArticolo, VolumeByMP, (QuantityMPImpiegata/VolumeByMP) as Impiego, CostoUnitario, ImportoCostoTOTALE
from ConsumiBudgetUnitByMP natural join buVol''')
consumiConsuntivoUnitByMP

,CodiceMP,NrArticolo,VolumeByMP,Impiego,CostoUnitario,ImportoCostoTOTALE
0,ART0000771,ART0005136,1.0,1.000000,230.950000,230.95
1,ART0000809,ART0005136,2.0,1.000000,561.350000,1122.70
2,ART0001066,ART0003099,1064.0,1.000000,0.001870,1.99
3,ART0001068,ART0004671,190.0,1.000000,0.036000,6.84
4,ART0001073,ART0002240,296.0,0.621622,0.086685,15.95
...,...,...,...,...,...,...
1074,ART0007130,ART0001228,15.0,1.000000,1.500000,22.50
1075,ART0007133,ART0005136,3.0,1.000000,0.000000,0.00
1076,ART0007141,ART0002191,2.0,1.000000,6.500000,13.00
1077,ART0007148,ART0002619,2.0,1.000000,30.000000,60.00


### Calcolo impiego risorse con volumi del budget

In [ ]:
sqldf('''
    select NrArticolo, NrOrdineProduzione, Descrizione, ''')

In [ ]:
impiegoBudgetNew

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,51.00,2.0
2,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,68.00,2.0
3,ART0000041,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,4.0
4,ART0000041,BUDGET,ODP17-05342,Fresatura,A30,RIS7,1.25,51.00,4.0
...,...,...,...,...,...,...,...,...,...
2006,ART0005136,BUDGET,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,12.75,10.0
2007,ART0005136,BUDGET,ODP17-15077,Tornitura,A20,RIS14,1.25,51.00,10.0
2008,ART0005136,BUDGET,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,42.50,1.0
2009,ART0005136,BUDGET,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,12.75,1.0
